# Filtracja bilateralna

## Konwolucja obrazu z filtrem o zadanych współczynnikach

Splot (konwolucję) obrazu wejściowego $I$ z filtrem $\psi$ dla ustalonego punktu obrazu $\mathbf{x}$ można przedstawić następująco:

\begin{equation}
\hat{I}(\mathbf{x}) = \frac{1}{W_N}\sum_{\mathbf{p} \in \eta(\mathbf{x})} \psi(||\mathbf{p}-\mathbf{x}||)I(\mathbf{p})
\end{equation}

gdzie:
- $\hat{I}$ jest obrazem wynikowym (przefiltrowanym),
- $W_N = \sum_y \psi(y)$ jest parametrem normalizującym współczynniki filtra $\psi$,
- $||\cdot||$ jest odległością między punktami obrazu $\mathbf{x}$ i $\mathbf{p}$ według ustalonej metryki (np. norma $L_2$). Uwaga, proszę pamiętać, że zarówno $\mathbf{x}$, jak i $\mathbf{p}$ to współrzędne przestrzenne,
- $\eta(\mathbf{x})$ jest otoczeniem punktu $\mathbf{x}$.

Funkcja $\psi$ decyduje o charakterze filtracji. Dla filtru Gaussowskiego:

\begin{equation}
\psi(y) = G_{\delta_s}(y)
\end{equation}

gdzie: $G_{\delta_s}(y)$ jest funkcją Gaussa z parametrem skali $\delta_s$.

Opisaną powyżej filtrację realizowaliśmy w ramach ćwiczenia "Przetwarzanie wstępne. Filtracja kontekstowa."

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np
from scipy import signal
from scipy.io import loadmat
import math

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/'

fileNames = ["MR_data.mat"]
for fileName in fileNames:
  if not os.path.exists(fileName):
      r = requests.get(url + fileName, allow_redirects=True)
      open(fileName, 'wb').write(r.content)

In [ ]:
def compare_imgs(imgs, titles=None):
    _, axs = plt.subplots(1, len(imgs), figsize=(20, 20))
    for i, img in enumerate(imgs):
        axs[i].imshow(img, 'gray')
        axs[i].set_title(titles[i] if titles else '')
        axs[i].axis('off')
    plt.show()

## Filtracja bilateralna

Wadą klasycznego splotu jest brak adaptacji współczynników filtra do lokalnego otoczenia $\eta(\mathbf{x})$ filtrowanego punktu $\mathbf{x}$.
Oznacza to wykorzystanie tych samych współczynników filtra $\psi$ niezależnie od tego czy otoczenie jest względnie jednorodne lub zawiera krawędzie obiektów (w tym przypadku dochodzi do "rozmywania" krawędzi).
Filtracja bilateralna uwzględnia lokalne otoczenie filtrowanego punktu, w ten sposób, że parametry filtra zmieniają się w zależności od "wyglądu" otocznia.


Współczynniki filtra obliczane są na podstawie odległości filtrowanego punktu $\mathbf{x}$ od każdego punktu otoczenia $\mathbf{p}$ w dziedzinie przestrzennej obrazu (tak jak przy typowym filtrze np. Gaussa) oraz odległości punktów w przeciwdziedzinie obrazu (np. na podstawie różnicy w jasności pikseli dla obrazu w odcieniach szarości):

\begin{equation}
\hat{I}(\mathbf{x}) = \frac{1}{W_N}\sum_{\mathbf{p} \in \eta(\mathbf{x})} \psi(||\mathbf{p}-\mathbf{x}||) \gamma(|I(\mathbf{p}) - I(\mathbf{x})|) I(\mathbf{p})
\end{equation}
gdzie:
- $W_N$ jest współczynnikiem normalizującym filtr,
- $\gamma$ jest funkcją odległości w przeciwdziedzinie obrazu, np. $\gamma(y)=\exp(-\frac{y^2}{2\delta_r^2})$
- parametr $\delta_r$ jest utożsamiany z poziomem szumu w obrazie i należy go dobrać w sposób empiryczny.

Proszę chwilę zastanowić się nad powyższym równaniem, w szczególności nad funkcją $\gamma$. Proszę wyznaczyć, jaka będzie wartość funkcji dla pikseli podobnych (różnica 0, 1, 2), a skrajnie różnych (255, 200).

##  Realizacja ćwiczenia

### Wczytanie danych

1. Wczytaj dane z pliku *MR_data.mat*. W tym celu wykorzystaj funkcję `loadmat` z pakietu scipy:
        from scipy.io import loadmat
        mat = loadmat('MR_data.mat')

In [ ]:
mat = loadmat('MR_data.mat')

2. Wczytany plik zawiera 5 obrazów: *I_noisefree*, *I_noisy1*, *I_noisy2*, *I_noisy3* oraz *I_noisy4*. Odczytać je można w następujący sposób:  
        Input = mat['I_noisy1']

In [ ]:
img_names = ['I_noisy1', 'I_noisy2', 'I_noisy3', 'I_noisy4']
imgs = [mat[img_name] for img_name in img_names]

3. Wyświetl wybrany obraz z pliku *MR_data.mat*. Zagadka - co to za obrazowanie medyczne?

In [ ]:
compare_imgs(imgs)

### "Klasyczna" konwolucja

1. Zdefiniuj parametry filtra Gaussowskiego: rozmiar okna i wariancję $\delta_S$.
2. Oblicz współczynniki filtra na podstawie zdefiniowanych parametrów (najprościej w ramach podwójnej pętli for).
2. Sprawdź ich poprawność i zwizualizuj filtr (tak jak w ćwiczeniu pt. "Przetwarzanie wstępne. Filtracja kontekstowa.").
3. Wykonaj kopię obrazu wejściowego: `IConv = Input.copy()`
4. Wykonaj podwójną pętlę po obrazie. Pomiń ramkę, dla której nie jest zdefiniowany kontekst o wybranej wielkości.
5. W każdej iteracji stwórz dwuwymiarową tablicę zawierającą aktualny kontekst.
6. Napisz funkcję, która będzie obliczała nową wartość piksela.
Argumentem tej funkcji są aktualnie przetwarzane okno i współczynniki filtra.
7. Obliczoną wartość przypisz do odpowiedniego piksela kopii obrazu wejściowego.
8. Wyświetl wynik filtracji.
9. Porównaj wynik z obrazem oryginalnym.

In [ ]:
def mesh(fun):
    """Funkcja do wizualizacji filtra gausowskiego"""
    fig = plt.figure()
    ax = fig.add_subplot(projection = '3d')

    size = int(fun.shape[0]//2)
    X = np.arange(-size, size+1, 1)
    Y = np.arange(-size, size+1, 1)
    X, Y = np.meshgrid(X, Y)
    Z = fun

    ax.plot_surface(X, Y, Z)
    plt.show()

def fgaussian(size, sigma):
    """Funkcja do obliczania współczynnika filtra Gaussowskiego"""
    m = n = size
    h, k = m//2, n//2
    x, y = np.mgrid[-h : h+1, -k : k+1]
    g = np.exp(-(x**2 + y**2)/(2 * sigma**2))
    return g / g.sum()

def gamma(y, delta_r):
    """Funkcja do obliczania współczynnika funkcji odległości w przeciwdziedzinie obrazu"""
    return np.exp(-y ** 2 / (2 * delta_r**2))

In [ ]:
def gaussian_filter(img, size_, delta_s):
    """Implementacja filtracji Gaussowskiej"""
    size = int(size_//2)
    img_filtered = np.zeros_like(img)
    
    gaussian_filter = fgaussian(size_, delta_s)
    # mesh(gaussian_filter)
    
    def get_new_value(x, y):
        window = img[x-size: x + size+1, y-size: y + size+1]
        return np.sum(window * gaussian_filter)

    for x in range(size, img_filtered.shape[0]-size):
        for y in range(size, img_filtered.shape[1]-size):
            img_filtered[x, y] = get_new_value(x, y)

    return np.uint8(img_filtered)

In [ ]:
# Wczytanie danych
img = imgs[0]

# Parametry filtra Gaussowskiego
size_ = 5
delta_s = 1

# Parametry filtracji bilateralnej
delta_r = 20

# Wykonanie filtracji bilateralnej
filtered_image = gaussian_filter(img, size_, delta_s)

# Wyświetlenie wyników
compare_imgs([img, filtered_image])


### Filtracja bilateralna

1. Zdefiniuj dodatkowy parametr: wariancję $\delta_R$.
3. Wykonaj kopię obrazu wejściowego: `IBilateral = Input.copy()`
4. Wykonaj podwójną pętlę po obrazie. Pomiń ramkę, dla której nie jest zdefiniowany kontekst o wybranej wielkości.
5. W każdej iteracji stwórz dwuwymiarową tablicę zawierającą aktualny kontekst.
6. Napisz funkcję, która będzie obliczała nową wartość piksela.
Argumentami funkcji są aktualnie przetwarzane okno, współczynniki filtra gausowskiego (takie same jak wcześniej) i wariancja $\delta_R$.
7. Oblicz odległość w przeciwdziedzinie (dla wartości pikseli).
8. Oblicz funkcję Gaussa dla obliczonych odległości z zadanym parametrem.
9. Wykonaj normalizację obliczonych współczynników.
10. Obliczoną wartość przypisz do odpowiedniego piksela kopii obrazu wejściowego.
11. Wyświetl wynik filtracji.
12. Porównaj wynik z obrazem oryginalnym.

In [ ]:
def bilateral_filter(img, size_, delta_s, delta_r):
    """Implementacja filtrowania bilateralnego"""
    size = int(size_//2)
    img_filtered = np.zeros_like(img)
    
    gaussian_filter = fgaussian(size_, delta_s)
    # mesh(gaussian_filter)

    def get_new_value(x, y):
        window = img[x - size: x + size + 1, y - size: y + size + 1]

        filter_window = gaussian_filter * gamma(np.abs(window - img[x, y]), delta_r)
        normalized_filter = filter_window / np.sum(filter_window)

        return np.sum(window * normalized_filter)
    
    for x in range(size, img_filtered.shape[0]-size):
        for y in range(size, img_filtered.shape[1]-size):
            img_filtered[x, y] = get_new_value(x, y)
    
    return np.uint8(img_filtered)

In [ ]:
# Wczytanie danych
img = imgs[0]

# Parametry filtra Gaussowskiego
size = 5
delta_s = 1

# Parametry filtrowania bilateralnego
delta_r = 15

# Wykonanie filtrowania bilateralnego
filtered_image_bilateral = bilateral_filter(img, size, delta_s, delta_r)

# Wyświetlenie wyników
compare_imgs([img, filtered_image_bilateral])

In [ ]:
img = imgs[2]
size = 5
deltas_s = [0.1, 0.5, 1, 2, 5]
deltas_r = [5, 10, 20, 30, 50]

for delta_s in deltas_s:
    for delta_r in deltas_r:
        filtered_image_gaussian = gaussian_filter(img, size, delta_s)
        filtered_image_bilateral = bilateral_filter(img, 5, delta_s, delta_r)
        compare_imgs([img, filtered_image_gaussian, filtered_image_bilateral], ['otiginal',f'delta_s={delta_s}', f'delta_r={delta_r}, delta_s={delta_s}'])